In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tabula.io as tabula
from IPython.display import display,HTML,Javascript
import ipywidgets as widgets
import regex as re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
#from googlesearch import search
from lxml import etree
import time
from duckduckgo_search import ddg
import os  
from matplotlib import pyplot as plt

# Download NLTK resources if not already installed
nltk.download('punkt')
nltk.download('stopwords')

# Initialize Porter Stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Load spaCy model and define preprocessing function
# nlp = spacy.load("en_core_web_sm")



[nltk_data] Downloading package punkt to C:\Users\Mahmud
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mahmud
[nltk_data]     Hasan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Singapore Polytechnic (SP)

In [2]:
# Function to clean course URLs and extract descriptions
def url_course_desc_scrap(df):
    # Lists to store cleaned URLs and course descriptions
    course_descriptions = []
    career_prospects = []

    for index, row in df.iterrows():
        print(row['Course Name'])
        clean_url = row['Course URL']
        #cleaned_urls.append(clean_url)
        career_prospect = extract_career_prospects(clean_url)
        career_prospects.append(career_prospect)
        try:
            webpage = requests.get(clean_url)
            soup = BeautifulSoup(webpage.content, "html.parser")
            dom = etree.HTML(str(soup))
            # Define a regex pattern to match HTML tags
            html_tag_pattern = re.compile(r'<[^>]*>')
            # Use XPath to select the parent <p> element that contains the <strong> element
            paragraph_with_strong = dom.xpath('/html/body/form/div[4]/div/div[2]/div[2]/div/div[1]')

            if paragraph_with_strong:
                # Extract the text from the selected <p> element and its children
                paragraph_text = etree.tostring(paragraph_with_strong[0], method='html').decode()
                cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                course_descriptions.append(cleaned_text)
                print("Success : course_descriptions ")
            else:
                course_descriptions.append("Course description was not found")
                print("Failed : course_descriptions ")
        except:
            course_descriptions.append("Course description was not found")
            print("Failed : course_descriptions ")            
                
    # Add the cleaned URLs and course descriptions to the DataFrame
    #df["Course URL"] = cleaned_urls
    df["Course Description"] = course_descriptions
    df["Career Prospect"] = career_prospects
    
# Function to extract career prospects using XPath
def extract_career_prospects(url):
           
    career_prospects = []
    # Split the URL by '/' and keep only the first 7 components

    clean_url = url + "/your-future"
    #print(clean_url)
    try: 
        webpage = requests.get(clean_url)
        dom = etree.HTML(webpage.text)

        # Use the full XPath to select the element containing career prospects
        career_prospects_elements = dom.xpath("/html/body/form/div[4]/div/div[2]/div[2]/div/div[1]/div/div")

        if career_prospects_elements:
            # Extract the text from the selected element
            career_prospects_text = ", ".join(career_prospects_elements[0].itertext())
            #print(career_prospects_text)
            career_prospects.append(career_prospects_text)
            print("Success : Career prospect ")
        else:
            career_prospects.append("No element Found")
            print("Failed  : Career Prsopect ")
            
        return ", ".join(career_prospects)
    except:
        print("Failed  : Career Prsopect ")
        return "No element Found"
    
def sp_scrap():
    try:
        # URL of the website to scrape
        url = "https://www.sp.edu.sg/sp/admissions/admissions-exercises/admission-criteria/course-intake-and-jae-elr2b2"

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the table element you want to scrape
            table = soup.find('table')

            # Initialize an empty list to store the scraped data
            data = []

            # Initialize a variable to store the current course category
            current_category = None

            # Find all rows (tr elements) in the table
            rows = table.find_all('tr')

            # Loop through each row and extract the data from the cells (td elements)
            for row in rows:
                # Check if this row represents a course category
                if row.find('th', colspan="3"):
                    current_category = row.get_text(strip=True)
                else:
                    # Find all cells (td elements) in the row
                    cells = row.find_all('td')
                    
                    # Check if the row has enough cells (at least 3)
                    if len(cells) >= 3:
                        # Extract text from each cell and store it in a list
                        row_data = [current_category] + [cell.get_text(strip=True) for cell in cells]

                        # Extract the course URL if available
                        course_link = cells[0].find('a', href=True)
                        if course_link:
                            course_url = "https://www.sp.edu.sg" + course_link['href']
                        else:
                            course_url = None

                        # Append the course URL to the row data
                        row_data.append(course_url)

                        # Append the row data to the main data list
                        data.append(row_data)

            # Remove empty rows (rows with no data)
            data = [row for row in data if row]

            # Add the name of the university before each row
            university_name = "Singapore Polytechnic"  # Change this to the actual name if needed
            data_with_university = [[university_name] + row for row in data]

            # Convert the data into a DataFrame
            df = pd.DataFrame(data_with_university, columns=["University", "Course Category", "Course Info", "2023 Planned Intake", "Range of Net 2023 JAE ELR2B2", "Course URL"])

            # Split the "Course Info" column into "Course Name" and "Course Code"
            df[['Course Name', 'Course Code']] = df['Course Info'].str.split("(", n=1, expand=True)

            # Remove everything starting from the ")" character in the "Course Code" column
            df['Course Code'] = df['Course Code'].str.split(')', n=1).str[0]

            # Drop the original "Course Info" column
            df.drop(columns=['Course Info'], inplace=True)
            new_df = df[['University', 'Course Category', 'Course Name', 'Course Code', '2023 Planned Intake', "Range of Net 2023 JAE ELR2B2", "Course URL"]]
            new_df = pd.DataFrame(new_df)
            url_course_desc_scrap(new_df)
            # Print the DataFrame
            #display(new_df)
            #new_df = new_df[new_df["University","Course Category","Course Name","Course Code","2023 Planned Intake","Range of Net 2023 JAE ELR2B2", "Course URL","Course Description","Career Prospect"]]
            new_df.to_csv("SP_scrap.csv",index = False)
            return new_df
        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
    except Exception as e:
        print(e)
# Debug

try:
    sp= pd.read_csv('SP_scrap.csv') 
    display(sp.columns)
    print("SP Course data loaded from storage")
except: 
    print("SP Course data Scraped from internet")
    sp  = sp_scrap()
    display(sp)

Index(['University', 'Course Category', 'Course Name', 'Course Code',
       '2023 Planned Intake', 'Range of Net 2023 JAE ELR2B2', 'Course URL',
       'Course Description', 'Career Prospect'],
      dtype='object')

SP Course data loaded from storage


# TP

In [3]:
def extract_career_prospects(url):
    # Send a GET request to the web page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page with lxml
        html_content = response.text
        dom = etree.HTML(html_content)

        # Use the specified XPath to select the element
        target_element = dom.xpath('/html/body/div[3]/div[1]/div/div[3]/div/div/div[2]/div/div[11]/div/div[3]/div')
        
        # Find all elements with class="full-tile-alignment"
        soup = BeautifulSoup(response.text, "html.parser")
        full_tile_elements = soup.find_all(class_="cmp-flip-tile__box")
        
        fulltext = []
        if target_element:
            # Extract the text from the selected element
            text = "".join(target_element[0].itertext())

            # Remove newlines from the text while preserving other spaces
            text = re.sub(r'\n', '', text)

            text = text.replace('\t', '')
            text = re.sub(r'\r\r', ': ', text)
            text = re.sub(r'\r', ',', text)
            # print(text)
            fulltext.append(text)
            return  ", ".join(fulltext)
            print("Success: Career")
        
        elif full_tile_elements:
            # Loop through each element and extract the text
            for element in full_tile_elements:
                text = element.get_text()
                # print(text)

                # Remove newlines from the text while preserving other spaces
                text = re.sub(r'\n\n', '', text)
                fulltext.append(text)
            print("Success: Career")
            return  ", ".join(fulltext)
        
        
        else:
            print("failed: Career")
            return "class not found"
            
    else:
        print("Failed: Career")
        return "Failed to retrieve the web page."
        
# Function to clean course URLs and extract descriptions
def url_course_desc_scrap(df):
    # Lists to store cleaned URLs and course descriptions
    #cleaned_urls = []
    course_descriptions = []
    career_prospects = []
    for index, row in df.iterrows():
        # Search for the course URL
        #code = row['Course Code']
        course_url = row['Course URL']
        #cleaned_urls.append(course_url)
        career_prospects.append(extract_career_prospects(course_url))
        webpage = requests.get(course_url)

            
        if webpage.status_code == 200:
            soup = BeautifulSoup(webpage.content, "html.parser")
            dom = etree.HTML(str(soup))
            # Define a regex pattern to match HTML tags
            html_tag_pattern = re.compile(r'<[^>]*>')
            # Use XPath to select the parent <p> element that contains the <strong> element
            paragraph_with_strong = dom.xpath('/html/body/div[3]/div[1]/div/div[3]/div/div/div[2]/div/div[3]/div/div')

            if paragraph_with_strong:
                # Extract the text from the selected <p> element and its children
                paragraph_text = etree.tostring(paragraph_with_strong[0], method='html').decode()
                cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                course_descriptions.append(cleaned_text)
                print("Success: Description")
            else:
                print("Failed: Description")
                course_descriptions.append("Course description was not found")
        else :
            #cleaned_urls.append(course_url)
            print("Failed: Description")
            course_descriptions.append("Course URL not found")
    df["Course Description"] = course_descriptions
    df["Career Prospect"] = career_prospects
# Function to extract career prospects using XPath

def tp_scrap():
    # URL of the website to scrape
    url = "https://www.tp.edu.sg/admissions-and-finance/course-intake-aggregate-range.html"
    
    # Initialize empty lists to store data
    course_data = []
    
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        current_category = None

        # Find all tables in the HTML
        tables = soup.find_all('table')

        # Loop through each table
        for table in tables:
            # Find the preceding <b> tag to determine the course category
            category_tag = table.find_previous('b')
            if category_tag:
                category = category_tag.get_text(strip=True)

                # Find all rows (tr elements) in the table
                rows = table.find_all('tr')

                # Loop through each row and extract the data from the cells (td elements)
                for row in rows[1:]:  # Skip the header row
                    cells = row.find_all('td')
                    # Extract text from each cell and store it in a list
                    course_name = cells[0].find('a').get_text(strip=True)
                    # print(course_name)
                    planned_intake = cells[1].get_text(strip=True)
                    elr2b2_range = cells[2].get_text(strip=True)
                    course_url = "https://www.tp.edu.sg" +cells[0].find('a')['href']
                    university_name = "Temasek Polytechnic"  # Change this to the actual name if needed

                    # Append the data to the course_data list
                    course_data.append([university_name, category, course_name, planned_intake, elr2b2_range, course_url])

        df = pd.DataFrame(course_data, columns=["University","Course Category", "Diploma", 
                                                "2023 Planned Intake", "Range of Net 2023 JAE ELR2B2", "Course URL"])
        # Split the "Diploma" column into "Course Name" and "Course Code"
        df[['Course Name', 'Course Code']] = df['Diploma'].str.split("(", n=1, expand=True)

        # Remove everything starting from the ")" character in the "Course Code" column
        df['Course Code'] = df['Course Code'].str.split(')', n=1).str[0]
        df.drop_duplicates(subset=['Diploma'])

        # Drop the original "Diploma" column
        df.drop(columns=['Diploma'], inplace=True)

        # Now select the desired columns and reorder them
        df = df[["University", "Course Category", "Course Name", "Course Code", "2023 Planned Intake", "Range of Net 2023 JAE ELR2B2", "Course URL"]]

        # Call the function to extract course descriptions and career prospects
        newdf = df.tail(41)
        url_course_desc_scrap(newdf)

        # Add the "Course Description" and "Career Prospect" columns
        newdf = newdf[["University", "Course Category", "Course Name", "Course Code", "2023 Planned Intake", "Range of Net 2023 JAE ELR2B2", "Course URL", "Course Description", "Career Prospect"]]

        # Save the DataFrame to a CSV file
        newdf.to_csv("TP_scrap.csv", index=False)
        return newdf
                         
        
# Debug
try:
    tp= pd.read_csv('TP_scrap.csv') 
    display(tp)
    print("TP Course data loaded from storage")
except: 
    print("TP Course data Scraped from internet")
    tp  = tp_scrap()
    display(tp)


,University,Course Category,Course Name,Course Code,2023 Planned Intake,Range of Net 2023 JAE ELR2B2,Course URL,Course Description,Career Prospect
0,Temasek Polytechnic,School of Applied Science,Chemical Engineering,T33,100,5 - 14,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nChemical engineers play a k...,Process Technicians\nExecutes safe and reliab...
1,Temasek Polytechnic,School of Applied Science,Common Science Programme,T70,150,7 - 12,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nAre you passionate about sc...,...
2,Temasek Polytechnic,School of Applied Science,"Food, Nutrition & Culinary Science",T26,50,8 - 10,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nHave you ever wanted to exp...,Assistant Food Technologist\nPrepares equipmen...
3,Temasek Polytechnic,School of Applied Science,Medical Biotechnology,T64,65,3 - 9,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nHave you wondered about the...,Laboratory Assistant or Technical Support Offi...
4,Temasek Polytechnic,School of Applied Science,Pharmaceutical Science,T25,70,5 - 10,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nMedicines affect our qualit...,Pharmacy Technician in Hospital and Retail Pha...
5,Temasek Polytechnic,School of Applied Science,Veterinary Technology,T45,45,5 - 9,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nTurn your love for animals ...,Veterinary Technician \nAssists veterinarians ...
6,Temasek Polytechnic,School of Business,Accountancy & Finance,T02,100,5 - 11,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nSeize the opportunity to th...,class not found
7,Temasek Polytechnic,School of Business,Business,T10,165,5 - 13,https://www.tp.edu.sg/schools-and-courses/stud...,"\nCourse Overview\nThrough this course, you wi...",...
8,Temasek Polytechnic,School of Business,Common Business Programme,T01,290,6 - 15,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nOur one-year Common Busines...,class not found
9,Temasek Polytechnic,School of Business,Communications & Media Management,T40,85,6 - 13,https://www.tp.edu.sg/schools-and-courses/stud...,\nCourse Overview\nThis course exposes you to ...,...


TP Course data loaded from storage


# Republic Polytechnic (RP)

In [4]:
# Sample DataFrame with course details and URLs
def search_course_url(row):
    query = f"Diploma in {row['Course Name']}({row['Course Code']}) {row['University']}"
    print(query)
    try:
        num_page = 1
        search_results = ddg(query, max_results=1)
        if search_results:
            #print(search_results)
            print(search_results[0]['href'])
            return search_results[0]['href']

    except Exception as e:
        print(e)
        return "No results found"


# Function to clean course URLs and extract descriptions
def url_course_desc_scrap(df):
    # Lists to store cleaned URLs, course descriptions, and career prospects
    cleaned_urls = []
    course_descriptions = []
    career_prospects = []

    for index, row in df.iterrows():
        # Search for the course URL
        course_url = search_course_url(row)

        if course_url == "No results found":
            cleaned_urls.append(course_url)
            course_descriptions.append("Course URL not found")
            career_prospects.append("Course URL not found")
        else:
            # Split the URL by '/' and keep only the first 7 components

            cleaned_urls.append(course_url)

            webpage = requests.get(course_url)

            if webpage.status_code == 200:
                soup = BeautifulSoup(webpage.content, "html.parser")
                dom = etree.HTML(str(soup))

                # Define a regex pattern to match HTML tags
                html_tag_pattern = re.compile(r'<[^>]*>')

                # Use XPath to select the parent <p> element that contains the <strong> element for course descriptions
                description = dom.xpath('/html/body/form/div[3]/div[2]/div/div/div[1]/div[3]/div/div/div[1]/p')
                if description:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(description[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    course_descriptions.append(cleaned_text)
                    print("Success: Description")
                else:
                    print("Failed: Description")
                    course_descriptions.append("Course description was not found")

                # Use XPath to select the parent <p> element that contains the career prospects
                prospect = dom.xpath("/html/body/form/div[3]/div[2]/div/div/div[1]/div[3]/div/div/div[4]/div")
                if prospect:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(prospect[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    career_prospects.append(cleaned_text)
                    print("Success: Career")
                else:
                    career_prospects.append("Career prospects were not found")
            else:
                print("Failed: Career")
                cleaned_urls.append(course_url)
                course_descriptions.append("Course URL not found")
                career_prospects.append("Career prospects were not found")

    # Add the cleaned URLs, course descriptions, and career prospects to the DataFrame
    df["Course URL"] = cleaned_urls
    df["Course Description"] = course_descriptions
    df["Career Prospect"] = career_prospects
    
def rp_scrap():
    try:
        # URL of the website to scrape
        url = "https://www.rp.edu.sg/admissions/course-intake-numbers"

        # University name
        university_name = "Republic Polytechnic"

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all tables on the page
            tables = soup.find_all('table')

            # Initialize an empty list to store data from all tables
            all_data = []

            # Loop through each table
            for table in tables:
                # Initialize an empty list to store data from the current table
                data = []

                # Find all rows (tr elements) in the table
                rows = table.find_all('tr')

                # Loop through each row and extract the data from the cells (td elements)
                for row in rows:
                    # Find all cells (td elements) in the row
                    cells = row.find_all('td')

                    # Extract text from each cell and store it in a list
                    row_data = [cell.get_text(strip=True) for cell in cells]

                    # Append the row data to the current table's data list
                    data.append(row_data)

                # Remove empty rows (rows with no data)
                data = [row for row in data if row]



                # Extract the course category from the table (assuming it's in the h3 header)
                course_category = table.find_previous('h3').text.strip()

                # Add the course category to each row
                data_with_category = [[course_category] + row for row in data]
                
                # Add the name of the university before each row
                data_with_university = [[university_name] + row for row in data_with_category]
                # Convert the data into a DataFrame for the current table
                df = pd.DataFrame(data_with_university, columns=['University','Course Category',  "Course Name", "Course Code", "Aggregate Type", "2023 Planned Intake", "Range of Net 2023 JAE ELR2B2"])
                
                # Drop the original "Course Info" column
                df.drop(columns=['Aggregate Type'], inplace=True)

                # Drop the original "Aggregate Type" column
                #df.drop(columns=['Category'], inplace=True)

                # Append the current table's data to the list of all data
                all_data.append(df)

            # Combine data from all tables into a single DataFrame
            combined_df = pd.concat(all_data, ignore_index=True)
            #combined_df = combined_df [combined_df["University","Course Category","Course Name","Course Code","2023 Planned Intake","Range of Net 2023 JAE ELR2B2"]]

            
            # Print the DataFrame
            url_course_desc_scrap(combined_df)
            #display(combined_df)
            combined_df.to_csv("RP_scrap.csv",index = False)
            return combined_df

        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
    except Exception as e:
        print(e)
# Debug 
try:
    rp= pd.read_csv('RP_scrap.csv') 
    display(rp)
    print("RP Course data loaded from storage")
except: 
    print("RP Course data Scraped from internet")
    rp  = rp_scrap()
    display(rp)


,University,Course Category,Course Name,Course Code,2023 Planned Intake,Range of Net 2023 JAE ELR2B2,Course URL,Course Description,Career Prospect
0,Republic Polytechnic,School of Applied Science,Biomedical Science,R14,98,7 to 12,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,Turn your fascination with the human body's re...,\n\nOption 1: Biomedical Research Track\n\nA24...
1,Republic Polytechnic,School of Applied Science,Biotechnology,R16,93,8 to 19,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,Understand new breakthroughs in science and ta...,\n\nOption 1: Biologics Track\n\nA103 Anatomy ...
2,Republic Polytechnic,School of Applied Science,Applied Chemistry,R17,51,12 to 18,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,From pharmaceutical drugs to skincare products...,\n\nOption 1: Materials Science Track\n\nA217 ...
3,Republic Polytechnic,School of Applied Science,Pharmaceutical Science,R22,152,8 to 19,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,Play a pivotal role in driving advances in mod...,\n\nOption 1: Industrial Pharmacy Track\n\nA36...
4,Republic Polytechnic,School of Applied Science,Common Science Programme,R59,194,12 to 18,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,Intrigued by the science behind everything?&#1...,\nFurther Studies\nUpon completion of the Comm...
5,Republic Polytechnic,School of Applied Science,Environmental & Marine Science,R62,64,8 to 15,https://www.rp.edu.sg/SAS/full-time-diplomas/D...,Be at the forefront of protecting the environm...,\n\nOption 1: Environmental Management and Tec...
6,Republic Polytechnic,School of Engineering,Industrial & Operations Management,R11,81,14 to 26,https://www.rp.edu.sg/SEG/full-time-diplomas/D...,Craft operational strategies that can transfor...,Career prospects were not found
7,Republic Polytechnic,School of Engineering,Supply Chain Management,R21,81,13 to 26,https://www.rp.edu.sg/SEG/full-time-diplomas/D...,The Diploma in Supply Chain\nManagement (DSCM)...,Career prospects were not found
8,Republic Polytechnic,School of Engineering,Aviation Management,R39,85,15 to 25,https://www.rp.edu.sg/SEG/full-time-diplomas/D...,\nAchieve your aspirations in the aviation ind...,Career prospects were not found
9,Republic Polytechnic,School of Engineering,Aerospace Engineering,R40,96,12 to 26,https://www.rp.edu.sg/SEG/full-time-diplomas/D...,Your runway to becoming a licensed Aircraft En...,Career prospects were not found


RP Course data loaded from storage


# Ngee Ann Polytechnic (NP)

In [5]:
def search_course_url(row):
    query = f"Diploma in {row['Course Name']}({row['Course Code']}) {row['University']}"
    #print(query)
    try:
        num_page = 1
        search_results = ddg(query, max_results=1)
        if search_results:
            #print(search_results)
            print(search_results[0]['href'])
            return search_results[0]['href']

    except Exception as e:
        print(e)
        return "No results found"


# Function to clean course URLs and extract descriptions
def url_course_desc_scrap(df):
    # Lists to store cleaned URLs, course descriptions, and career prospects
    cleaned_urls = []
    course_descriptions = []
    career_prospects = []

    for index, row in df.iterrows():
        # Search for the course URL
        course_url = search_course_url(row)

        if course_url == "No results found":
            print("Failed:: ALL")
            cleaned_urls.append(course_url)
            course_descriptions.append("Course URL not found")
            career_prospects.append("Course URL not found")
        else:
            # Split the URL by '/' and keep only the first 7 components

            cleaned_urls.append(course_url)

            webpage = requests.get(course_url)

            if webpage.status_code == 200:
                soup = BeautifulSoup(webpage.content, "html.parser")
                dom = etree.HTML(str(soup))

                # Define a regex pattern to match HTML tags
                html_tag_pattern = re.compile(r'<[^>]*>')

                # Use XPath to select the parent <p> element that contains the <strong> element for course descriptions
                description = dom.xpath('/html/body/main/section[2]/div/div[2]/div[2]/p[1]')
                if description:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(description[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    course_descriptions.append(cleaned_text)
                    print("Success: Description")
                else:
                    course_descriptions.append("Course description was not found")
                    print("Failed: Description")

                # Use XPath to select the parent <p> element that contains the career prospects
                prospect = dom.xpath("/html/body/main/section[2]/div/div[2]/div[2]/ul[2]")
                if prospect:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(prospect[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    career_prospects.append(cleaned_text)
                    print("Success: Prospect")
                    
                else:
                    print("Failed: Prospect")
                    career_prospects.append("Career prospects were not found")
            else:
                course_descriptions.append("Course URL not found")
                career_prospects.append("Career prospects were not found")

    # Add the cleaned URLs, course descriptions, and career prospects to the DataFrame
    df["Course URL"] = cleaned_urls
    df["Course Description"] = course_descriptions
    df["Career Prospect"] = career_prospects
def np_scrap():
    try:
        # URL of the website to scrape
        url = "https://www.np.edu.sg/admissions-enrolment/guide-for-prospective-students/elr2b2"

        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the table element you want to scrape
            table = soup.find('table')

            # Initialize an empty list to store the scraped data
            data = []
            
            # Initialize a variable to store the current course category
            current_category = None

            # Find all rows (tr elements) in the table
            rows = table.find_all('tr')

            # Loop through each row and extract the data from the cells (td elements)
            for row in rows:
                # Check if this row represents a course category
                if row.find('th', colspan="4"):
                    current_category = row.get_text(strip=True)
                else:
                    # Find all cells (td elements) in the row
                    cells = row.find_all('td')
                    
                    # Check if the row has enough cells (at least 3)
                    if len(cells) >= 3:
                        # Extract text from each cell and store it in a list
                        row_data = [current_category] + [cell.get_text(strip=True) for cell in cells]

                        # Extract the course URL if available
#                         course_link = cells[0].find('a', href=True)
#                         if course_link:
#                             course_url = "https://www.np.edu.sg" + course_link['href']
#                         else:
#                             course_url = None

#                         # Append the course URL to the row data
#                         row_data.append(course_url)

                        # Append the row data to the main data list
                        data.append(row_data)

            # Remove empty rows (rows with no data)
            data = [row for row in data if row]

            # Add the name of the university before each row
            university_name = "Ngee Ann Polytechnic"  # Change this to the actual name if needed
            data_with_university = [[university_name] + row for row in data]

             # Convert the data into a DataFrame for the current table
            df = pd.DataFrame(data_with_university, columns=["University","Course Category","Course", "Aggregate Type", "Range of Net 2023 JAE ELR2B2", "2023 Planned Intake"])

            # Step 1: Remove "New!" from the "Course" column
            df["Course"] = df["Course"].str.replace("New!", "").str.strip()

            # Step 2: Extract the last three characters and put them in "Course Code"
            df["Course Code"] = df["Course"].str[-3:]

            # Step 3: Put the remaining characters in "Course Name"
            df["Course Name"] = df["Course"].str[:-3].str.strip()
            df = df.drop(columns=['Course','Aggregate Type'])
            new_df = df[['University',"Course Category", 'Course Name', 'Course Code', '2023 Planned Intake', "Range of Net 2023 JAE ELR2B2"]]

            url_course_desc_scrap(new_df)
            
            # Print the DataFrame
            #display(new_df)
            new_df.to_csv("NP_scrap.csv",index = False)
            return new_df

        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
    except Exception as e:
         print(e)
# Debug 
try:
    np= pd.read_csv('NP_scrap.csv') 
    display(np)
    print("NP Course data loaded from storage")
except: 
    print("NP Course data Scraped from internet")
    np  = np_scrap()
    display(np)


,University,Course Category,Course Name,Course Code,2023 Planned Intake,Range of Net 2023 JAE ELR2B2,Course URL,Course Description,Career Prospect
0,Ngee Ann Polytechnic,Applied Sciences,Biomedical Science,N59,60,3 to 7,https://www.np.edu.sg/schools-courses/academic...,Fascinated by the structure of living organism...,Australian National UniversityMurdoch Universi...
1,Ngee Ann Polytechnic,Applied Sciences,Chemical & Biomolecular Engineering,N56,75,8 to 12,https://www.np.edu.sg/schools-courses/academic...,A broad-based course that integrates biologica...,The University of AdelaideThe University of Me...
2,Ngee Ann Polytechnic,Applied Sciences,Common Science Programme,N15,50,5 to 9,https://www.np.edu.sg/schools-courses/academic...,Want to make a difference in the world of scie...,Career prospects were not found
3,Ngee Ann Polytechnic,Applied Sciences,Environmental & Water Technology,N74,45,6 to 15,https://www.np.edu.sg/schools-courses/academic...,With life in Singapore set to be much greener ...,Murdoch UniversityThe University of AdelaideTh...
4,Ngee Ann Polytechnic,Applied Sciences,Landscape Design & Horticulture,N57,40,4 to 16,https://www.np.edu.sg/schools-courses/academic...,Want to be the creative drive behind Singapore...,Australian National UniversityMonash Universit...
5,Ngee Ann Polytechnic,Applied Sciences,Pharmaceutical Science,N73,50,3 to 9,https://www.np.edu.sg/schools-courses/academic...,Discover what it is like to work at the forefr...,Monash UniversityQueensland University of Tech...
6,Ngee Ann Polytechnic,Built Environment,Hotel & Leisure Facilities Management,N40,95,7 to 17,https://www.np.edu.sg/schools-courses/academic...,Smart technology is transforming the hotel and...,National University of Singapore&#160;Bachelor...
7,Ngee Ann Polytechnic,Business & Management,Accountancy,N51,135,4 to 12,https://www.np.edu.sg/schools-courses/academic...,"Beyond bookkeeping and budgeting, accounting w...",Business AdvisorBusiness Analyst&#160;Environm...
8,Ngee Ann Polytechnic,Business & Management,Arts Business Management,N91,40,5 to 12,https://www.np.edu.sg/schools-courses/academic...,"If you love the idea of planning arts events, ...",Edith Cowan University&#160;\n Bachelor...
9,Ngee Ann Polytechnic,Business & Management,Banking & Finance,N53,75,5 to 10,https://www.np.edu.sg/schools-courses/academic...,"Technology, innovation and sustainability are ...",Career prospects were not found


NP Course data loaded from storage


# Nanyang Polytechnic (NYP)

In [6]:
def search_course_url(row):
    query = f"full-time-courses {row['Course Name']} ({row['Course Code']}) {row['University']}"
    #print(query)
    try:
        num_page = 1
        search_results = ddg(query, max_results=1)
        if search_results:
            #print(search_results)
            #print(search_results[0]['href'])
            return search_results[0]['href']

    except Exception as e:
        print(e)
        return "No results found"


# Function to clean course URLs and extract descriptions
def url_course_desc_scrap(df):
    # Lists to store cleaned URLs, course descriptions, and career prospects
    cleaned_urls = []
    course_descriptions = []
    career_prospects = []

    for index, row in df.iterrows():
        # Search for the course URL
        course_url = search_course_url(row)

        if course_url == "No results found":
            cleaned_urls.append(course_url)
            course_descriptions.append("Course URL not found")
            career_prospects.append("Course URL not found")
            print("Failed: ALL")
        else:
            # Split the URL by '/' and keep only the first 7 components

            cleaned_urls.append(course_url)

            webpage = requests.get(course_url,headers = {'User-agent': 'your bot 0.1'})

            if webpage.status_code == 200:
                soup = BeautifulSoup(webpage.content, "html.parser")
                dom = etree.HTML(str(soup))

                # Define a regex pattern to match HTML tags
                html_tag_pattern = re.compile(r'<[^>]*>')

                # Use XPath to select the parent <p> element that contains the <strong> element for course descriptions
                description = dom.xpath('/html/body/div[1]/div[4]/main/div/div/div[2]/section/div/div[1]/div/div/ul')
                if description:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(description[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    course_descriptions.append(cleaned_text)
                    print("Success: Description")
                else:
                    course_descriptions.append("Course description was not found")
                    print("Failed: Description")

                # Use XPath to select the parent <p> element that contains the career prospects
                prospect = dom.xpath("/html/body/div[1]/div[4]/main/div/div/div[8]/section/ul/li[1]/div/div/ul")
                if prospect:
                    # Extract the text from the selected <p> element and its children
                    paragraph_text = etree.tostring(prospect[0], method='html').decode()
                    cleaned_text = re.sub(html_tag_pattern, '', paragraph_text)
                    career_prospects.append(cleaned_text)
                    print("Success: Prospect")
                else:
                    print("Failed: Prospect")
                    career_prospects.append("Career prospects were not found")
            else:
                print("Failed: Prospect+description")
                cleaned_urls.append(course_url)
                course_descriptions.append("Course URL not found")
                career_prospects.append("Career prospects were not found")

    # Add the cleaned URLs, course descriptions, and career prospects to the DataFrame
    df["Course URL"] = cleaned_urls
    df["Course Description"] = course_descriptions
    df["Career Prospect"] = career_prospects
    
def nyp_scrap():
    try: 
        # Create a dictionary to map courses to their respective categories
        course_category_mapping = {
            'APPLIED CHEMISTRY': 'APPLIED SCIENCES',
            'BIOLOGICS & PROCESS TECHNOLOGY': 'APPLIED SCIENCES',
            'CHEMICAL & PHARMACEUTICAL TECHNOLOGY': 'APPLIED SCIENCES',
            'COMMON SCIENCE PROGRAMME': 'APPLIED SCIENCES',
            'FOOD SCIENCE & NUTRITION': 'APPLIED SCIENCES',
            'PHARMACEUTICAL SCIENCE': 'APPLIED SCIENCES',
            'ARCHITECTURE': 'BUILT ENVIRONMENT',
            'ACCOUNTANCY & FINANCE': 'BUSINESS & MANAGEMENT',
            'BANKING & FINANCE': 'BUSINESS & MANAGEMENT',
            'BUSINESS MANAGEMENT': 'BUSINESS & MANAGEMENT',
            'COMMON BUSINESS PROGRAMME': 'BUSINESS & MANAGEMENT',
            'FOOD & BEVERAGE BUSINESS': 'BUSINESS & MANAGEMENT',
            'HOSPITALITY & TOURISM MANAGEMENT': 'BUSINESS & MANAGEMENT',
            'MASS MEDIA MANAGEMENT': 'BUSINESS & MANAGEMENT',
            'SPORT & WELLNESS MANAGEMENT': 'BUSINESS & MANAGEMENT',
            'BUSINESS & FINANCIAL TECHNOLOGY': 'BUSINESS & MANAGEMENT',
            'ADVANCED & DIGITAL MANUFACTURING': 'ENGINEERING',
            'AERONAUTICAL & AEROSPACE TECHNOLOGY': 'ENGINEERING',
            'AEROSPACE SYSTEMS & MANAGEMENT': 'ENGINEERING',
            'AI & DATA ENGINEERING': 'ENGINEERING',
            'BIOMEDICAL ENGINEERING': 'ENGINEERING',
            'COMMON ENGINEERING PROGRAMME': 'ENGINEERING',
            'ELECTRONIC & COMPUTER ENGINEERING': 'ENGINEERING',
            'ENGINEERING WITH BUSINESS': 'ENGINEERING',
            'INFOCOMM & MEDIA ENGINEERING': 'ENGINEERING',
            'NANOTECHNOLOGY & MATERIALS SCIENCE': 'ENGINEERING',
            'ROBOTICS & MECHATRONICS': 'ENGINEERING',
            'ORAL HEALTH THERAPY': 'HEALTH SCIENCES',
            'NURSING': 'HEALTH SCIENCES',
            'SOCIAL WORK': 'HUMANITIES',
            'APPLIED AI & ANALYTICS': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'COMMON ICT PROGRAMME': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'CYBERSECURITY & DIGITAL FORENSICS': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'INFOCOMM & SECURITY': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'GAME DEVELOPMENT & TECHNOLOGY': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'INFORMATION TECHNOLOGY': 'INFORMATION & DIGITAL TECHNOLOGIES',
            'ARCHITECTURE': 'MEDIA & DESIGN',
            'ANIMATION, GAMES & VISUAL EFFECTS': 'MEDIA & DESIGN',
            'COMMON DESIGN & MEDIA PROGRAMME': 'MEDIA & DESIGN',
            'COMMUNICATION & MOTION DESIGN': 'MEDIA & DESIGN',
            'EXPERIENTIAL PRODUCT & INTERIOR DESIGN': 'MEDIA & DESIGN',
        }


        # Specify the pages you want to extract tables from (for example, page 1)
        pages = [1,2]
        url = "https://www.nyp.edu.sg/content/dam/nyp/admissions/full-time-diploma/admission-exercise/intake-and-jae-elr2b2-points/intake-and-elr2b2.pdf"

        # Extract tables from the PDF file
        tables = tabula.read_pdf(url, pages=pages, multiple_tables=False)
        tables[0].dropna(inplace=True)

        tables[0].columns = ['Course Name', 'Course Code', '2023 Planned Intake', "Range of Net 2023 JAE ELR2B2"]
        tables[0]["University"] = "Nanyang Polytechnic"  # Adding a University column
        tables[0] = tables[0][['University', 'Course Name', 'Course Code', '2023 Planned Intake', "Range of Net 2023 JAE ELR2B2"]]
        
        # Reset the index
        tables[0].reset_index(drop=True, inplace=True)
        # Create a dictionary with the new row data
        new_row_data = {
            'University': 'Nanyang Polytechnic',
            'Course Name': 'NANOTECHNOLOGY & MATERIALS SCIENCE',
            'Course Code': 'C50',
            '2023 Planned Intake': 40,
            'Range of Net 2023 JAE ELR2B2': '10 to 15'
        }

        # Convert the dictionary to a DataFrame
        new_row_df = pd.DataFrame([new_row_data])
        
        # Concatenate the new row DataFrame with the existing combined_df_2
        tables[0] = pd.concat([ tables[0], new_row_df], ignore_index=True)
        
        # Apply the mapping to the 'Course Name' column in your DataFrame
        tables[0]['Course Category'] = tables[0]['Course Name'].map(course_category_mapping)
        
        tables[0] = tables[0][['University',"Course Category", 'Course Name', 'Course Code', '2023 Planned Intake', "Range of Net 2023 JAE ELR2B2"]]
        url_course_desc_scrap(tables[0])

        #display(tables[0])
        tables[0].to_csv("NYP_scrap.csv",index = False)
        return tables[0]
    except Exception as e:
        print(e)

# Debug         
try:
    nyp= pd.read_csv('NYP_scrap.csv') 
    display(nyp)
    print("NYP Course data loaded from storage")
except: 
    print("NYP Course data Scraped from internet")
    nyp  = nyp_scrap()
    display(nyp)


,University,Course Category,Course Name,Course Code,2023 Planned Intake,Range of Net 2023 JAE ELR2B2,Course URL,Course Description,Career Prospect
0,Nanyang Polytechnic,APPLIED SCIENCES,APPLIED CHEMISTRY,C45,23,4 to 10,https://www.nyp.edu.sg/schools/sas/full-time-c...,\nImpact lives by creating innovative chemical...,\nAnalytical Chemist\nAssociate Scientist\nFor...
1,Nanyang Polytechnic,APPLIED SCIENCES,BIOLOGICS & PROCESS TECHNOLOGY,C49,23,5 to 10,https://www.nyp.edu.sg/schools/sas/full-time-c...,\nShine at the frontline of cutting-edge biolo...,\nLaboratory Analyst\nManufacturing Biotechnol...
2,Nanyang Polytechnic,APPLIED SCIENCES,CHEMICAL & PHARMACEUTICAL TECHNOLOGY,C73,88,7 to 14,https://www.nyp.edu.sg/schools/sas/full-time-c...,\nCraft innovative and green solutions to impr...,\nAssistant Process Engineer\nLaboratory Analy...
3,Nanyang Polytechnic,APPLIED SCIENCES,COMMON SCIENCE PROGRAMME,C27,116,8 to 12,https://www.nyp.edu.sg/schools/sas/full-time-c...,Course description was not found,Career prospects were not found
4,Nanyang Polytechnic,APPLIED SCIENCES,FOOD SCIENCE & NUTRITION,C69,45,5 to 12,https://www.nyp.edu.sg/schools/sas/full-time-c...,\nImprove the well-being of the community thro...,\nDietetic Assistant\nFood Hygiene Officer\nFo...
5,Nanyang Polytechnic,APPLIED SCIENCES,PHARMACEUTICAL SCIENCE,C65,59,6 to 10,https://www.nyp.edu.sg/schools/sas/full-time-c...,\nFight against the pandemic and other disease...,\nClinical Research Coordinator\nPharmaceutica...
6,Nanyang Polytechnic,MEDIA & DESIGN,ARCHITECTURE,C38,54,7 to 15,https://www.nyp.edu.sg/schools/sdm/full-time-c...,Course description was not found,Career prospects were not found
7,Nanyang Polytechnic,BUSINESS & MANAGEMENT,ACCOUNTANCY & FINANCE,C98,69,6 to 12,https://www.nyp.edu.sg/schools/sbm/full-time-c...,\nHone your competencies in emerging areas tha...,Career prospects were not found
8,Nanyang Polytechnic,BUSINESS & MANAGEMENT,BANKING & FINANCE,C96,66,5 to 12,https://www.nyp.edu.sg/schools/sbm/full-time-c...,\nLearn from industry leaders such as DBS Bank...,Career prospects were not found
9,Nanyang Polytechnic,BUSINESS & MANAGEMENT,BUSINESS MANAGEMENT,C94,248,6 to 15,https://www.nyp.edu.sg/schools/sbm/full-time-c...,Course description was not found,Career prospects were not found


NYP Course data loaded from storage


# Scrap Historical Data

In [7]:
def remove_asterisk_and_convert(value):
    try:
        # Check if the value contains an asterisk
        if '*' in value:
            # Remove the asterisk and convert to an integer
            return int(value.replace('*', ''))
        else:
            # Convert to an integer directly
            return int(value)
    except ValueError:
        # Handle non-integer values (e.g., if the value cannot be converted)
        return None
    
def historical_scrap():
    # Replace the URL with the URL of your published Google Sheets document
    url = "https://docs.google.com/spreadsheets/d/14b5VLIr9W5teiOp0vCg86B6xxpFiwk8jeIzwbdJmWjU/pubhtml#"

    # Use pandas to read tables from the HTML URL
    tables = pd.read_html(url)

    # Initialize an empty list to store formatted tables
    formatted_tables = []

    # Iterate through the extracted tables (if there are multiple)
    for i, table in enumerate(tables):
        if i > 0:
            # Drop the specified columns
            columns_to_drop = [0, 3, 4, 16, 17, 18, 19]
            table = table.drop(columns=table.columns[columns_to_drop])

            # Filter rows based on the length of the values in the first column
            table = table[table.iloc[:, 0].str.len() <= 3]

            # Rename columns
            table.columns = ['Course Code', 'Course Name',"2023 Planned Intake", "Cutoff 2023", "Cutoff 2022", "Cutoff 2021",
                             "Cutoff 2020", "Cutoff 2019", "Cutoff 2018", "Cutoff 2017", "Cutoff 2016", "Cutoff 2015", "Cutoff 2014"]

            # Convert "Cutoff 2023" column to numeric, coerce non-numeric values to NaN
            table['2023 Planned Intake'] = pd.to_numeric(table['2023 Planned Intake'], errors='coerce')

            # Drop rows with non-integer "Cutoff 2023" values
            table = table.dropna(subset=['2023 Planned Intake'], how='any')
            #display(table)
            # Convert cutoff columns to numeric, handling non-numeric values as NaN
            cutoff_columns = ['Cutoff 2023', 'Cutoff 2022', 'Cutoff 2021', 'Cutoff 2020', 'Cutoff 2019',
                              'Cutoff 2018', 'Cutoff 2017', 'Cutoff 2016', 'Cutoff 2015', 'Cutoff 2014']
            # Apply the custom function to the column
            table[cutoff_columns] = table[cutoff_columns].applymap(remove_asterisk_and_convert)

            table[cutoff_columns] = table[cutoff_columns].apply(pd.to_numeric, errors='ignore')

            
            # Calculate mean, median, and mode for each row from 'Cutoff 2023' to 'Cutoff 2014'
            table['Mean Cutoff'] = table[cutoff_columns].mean(axis=1)
            table['Median Cutoff'] = table[cutoff_columns].median(axis=1)
            table['Mode Cutoff'] = table[cutoff_columns].mode(axis=1).iloc[:, 0]

            # Replace "Course Code" for "NAUTICAL STUDIES" with "DNS"
            table.loc[table['Course Name'] == 'NAUTICAL STUDIES (APPLY VIA DAE ONLY)', 'Course Code'] = 'DNS'

            # Append the formatted table to the list
            formatted_tables.append(table)

    # Concatenate the formatted tables into a single DataFrame
    combined_df_2 = pd.concat(formatted_tables, ignore_index=True)
    combined_df_2['Course Name'] = combined_df_2['Course Name'].str.title()
    
    display(combined_df_2)
    combined_df_2.to_csv("history_scrap.csv",index = False)
    return combined_df_2

# Display the combined DataFrame
# historical = historical_scrap()
# display(historical[historical['Course Code'] =='C97'])
# display(combined_df_2[combined_df_2['Course Code'] == 'DNS'])


# Combine The Scrapped Courses and Historical Data

In [8]:
def scrap_all_merge():
    combined_df_1 = pd.concat([sp,tp,rp,np,nyp], axis=0)
    combined_df_1.reset_index(drop=True, inplace=True)
    # Convert "Cutoff 2023" column to numeric, coerce non-numeric values to NaN
    combined_df_1['2023 Planned Intake'] = pd.to_numeric(combined_df_1['2023 Planned Intake'], errors='coerce')

    combined_df_1['Course Name'] = combined_df_1['Course Name'].str.title()

    #display(combined_df_1)
    combined_df_2 = historical_scrap()

    ## Perform an inner join based on the "Course Code" column
    merged_df = pd.merge(combined_df_1, combined_df_2, on="Course Code", how="left")

    # Drop duplicate columns (suffixes are added to duplicate columns)
    #merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    merged_df =merged_df.drop(columns=['Course Name_y','2023 Planned Intake_y'])

    merged_df = merged_df.rename(columns={'2023 Planned Intake_x': '2023 Planned Intake'})
    merged_df = merged_df.rename(columns={'Course Name_x': 'Course Name'})
    
    return merged_df
    # Display the DataFrame
    # display(merged_df)


In [9]:
try:
    all_courses= pd.read_csv('All_output.csv') 
    display(all_courses.columns)
    print("All Course data loaded from storage")
except: 
    print("Scrapping and creating new dataframe")
    all_courses = scrap_all_merge()
    #os.makedirs('folder/subfolder', exist_ok=True)  
    all_courses.to_csv('All_output.csv',index= False) 
    display(all_courses.columns)
    

Index(['University', 'Course Category', 'Course Name', 'Course Code',
       '2023 Planned Intake', 'Range of Net 2023 JAE ELR2B2', 'Course URL',
       'Course Description', 'Career Prospect', 'Cutoff 2023', 'Cutoff 2022',
       'Cutoff 2021', 'Cutoff 2020', 'Cutoff 2019', 'Cutoff 2018',
       'Cutoff 2017', 'Cutoff 2016', 'Cutoff 2015', 'Cutoff 2014',
       'Mean Cutoff', 'Median Cutoff', 'Mode Cutoff'],
      dtype='object')

All Course data loaded from storage


In [10]:
display(all_courses)

,University,Course Category,Course Name,Course Code,2023 Planned Intake,Range of Net 2023 JAE ELR2B2,Course URL,Course Description,Career Prospect,Cutoff 2023,...,Cutoff 2020,Cutoff 2019,Cutoff 2018,Cutoff 2017,Cutoff 2016,Cutoff 2015,Cutoff 2014,Mean Cutoff,Median Cutoff,Mode Cutoff
0,Singapore Polytechnic,APPLIED SCIENCES,Applied Chemistry,S64,75,4 to 9,https://www.sp.edu.sg/cls/courses/full-time-di...,\nDiploma in Applied Chemistry (DAPC - S64)Unv...,"Application Chemist, Assistant Engineer, Chemi...",9.0,...,10.0,10.0,10.0,10.0,10.0,11.0,11.0,9.9,10.0,10.0
1,Singapore Polytechnic,APPLIED SCIENCES,Biomedical Science,S98,75,3 to 7,https://www.sp.edu.sg/cls/courses/full-time-di...,\nDiploma in Biomedical Science (DBS - S98)The...,,7.0,...,8.0,7.0,7.0,7.0,7.0,8.0,8.0,7.3,7.0,7.0
2,Singapore Polytechnic,APPLIED SCIENCES,Chemical Engineering,S70,120,7 to 14,https://www.sp.edu.sg/cls/courses/full-time-di...,\nDiploma in Chemical Engineering (DCHE - S70)...,"Assistant Biotechnologist, Laboratory Analyst,...",14.0,...,13.0,13.0,13.0,13.0,12.0,12.0,12.0,12.8,13.0,13.0
3,Singapore Polytechnic,APPLIED SCIENCES,Common Science Programme,S28,105,3 to 10,https://www.sp.edu.sg/cls/courses/full-time-di...,\nCommon Science Programme (DCSP - S28)Discove...,No element Found,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0
4,Singapore Polytechnic,APPLIED SCIENCES,Food Science & Technology,S47,65,4 to 11,https://www.sp.edu.sg/cls/courses/full-time-di...,\nDiploma in Food Science &amp; Technology (DF...,",",11.0,...,13.0,12.0,13.0,13.0,14.0,14.0,16.0,13.1,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Nanyang Polytechnic,MEDIA & DESIGN,"Animation, Games & Visual Effects",C29,125,4 to 10,https://www.nyp.edu.sg/schools/sdm/full-time-c...,Course description was not found,Career prospects were not found,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0
196,Nanyang Polytechnic,MEDIA & DESIGN,Common Design & Media Programme,C28,132,4 to 14,https://www.nyp.edu.sg/schools/sdm/full-time-c...,Course description was not found,Career prospects were not found,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,14.0
197,Nanyang Polytechnic,MEDIA & DESIGN,Communication & Motion Design,C30,83,4 to 15,https://www.nyp.edu.sg/schools/sdm/full-time-c...,Course description was not found,Career prospects were not found,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,15.0,15.0
198,Nanyang Polytechnic,MEDIA & DESIGN,Experiential Product & Interior Design,C32,110,5 to 17,https://www.nyp.edu.sg/schools/sdm/full-time-c...,Course description was not found,Career prospects were not found,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17.0,17.0


# Apply Different Filtering approaches

## TF-IDF vectorizer Based approach (Applied)

In [11]:
def preprocess_text(text):
    # Remove special characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords and apply stemming
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    
    # Reconstruct the processed text
    processed_text = ' '.join(processed_tokens)
    
    return processed_text

def filter_courses(merged_df, user_interest, user_score, similarity_threshold=0.5):
    # Preprocess user input
    user_interest = preprocess_text(user_interest)

    # Combine "Course Name" and "Course Category" into a single column
    merged_df['Combined Course Info'] = merged_df['Course Name'] + ' ' + merged_df['Course Category']

    # Apply preprocessing to the 'Combined Course Info' column
    merged_df['Cleaned Combined Info'] = merged_df['Combined Course Info'].apply(preprocess_text)

    # Initialize TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the combined course information
    tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['Cleaned Combined Info'])

    # Transform the user's input
    user_interest_vector = tfidf_vectorizer.transform([user_interest])

    # Calculate cosine similarity between user input and combined course information
    cosine_similarities = cosine_similarity(user_interest_vector, tfidf_matrix).flatten()

    # Add courses with similarity above the threshold to filtered_df
    filtered_df = merged_df[cosine_similarities >= similarity_threshold]

    # Filter further based on user score
    filtered_df = filtered_df[filtered_df['Cutoff 2023'] >= user_score]

    return filtered_df


In [12]:
# user_interest ="arts"
# user_score = 3
# filer_course_1(all_courses, user_interest,user_score,similarity_threshold = 0.4)

## NLP based approach (Not Applied)

In [13]:
def preprocess_text(text):
    # Remove special characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords and apply stemming
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    
    # Reconstruct the processed text
    processed_text = ' '.join(processed_tokens)
    
    return processed_text

def filter_courses_2(merged_df,user_interest,similarity_threshold = 0.5):
    
    # Load the English language model
    nlp = spacy.load("en_core_web_sm")
    
    # Process the course names and user input
    merged_df['Course Name'] = merged_df['Course Name'].str.lower()
    user_interest = user_interest.lower()
    user_interest_doc = nlp(user_interest)

    # Tokenize and vectorize the user's input
    user_interest_vector = user_interest_doc.vector.reshape(1, -1)

    # Create a list of course name vectors
    course_name_vectors = []

    for course_name in merged_df['Course Name']:
        course_name_doc = nlp(course_name)
        course_name_vector = course_name_doc.vector.reshape(1, -1)
        course_name_vectors.append(course_name_vector)

    # Calculate cosine similarities
    similarities = cosine_similarity(np.vstack(course_name_vectors), user_interest_vector)

    filtered_df = pd.DataFrame(columns=merged_df.columns)
    # Filter courses with similarity scores above the threshold
    filtered_courses = merged_df[similarities.flatten() > similarity_threshold]

    # Add the filtered courses to the filtered_df
    filtered_df = pd.concat([filtered_df, filtered_courses])
    # display(filtered_df)
    # print(len(filtered_df))
    return filtered_df

In [14]:
# interest_of_studies_input= "Data Science and Machine Learning"
# filter_courses_2(all_courses,interest_of_studies_input,similarity_threshold = 0.8)

## Word Vectorized approach (Not applied)

In [15]:
def filter_courses_3(all_courses,user_input):

    # Load the Word2Vec model
    word_vectors = api.load("word2vec-google-news-300")

    # Function to preprocess text
    def preprocess_text(text):
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text

    # Sample user input (replace this with user input)

    user_input = preprocess_text(user_input)

    # Compute word embeddings for user input
    user_words = [word for word in user_input.split() if word in word_vectors]
    if user_words:
        user_vec = np.mean([word_vectors[word] for word in user_words], axis=0)
    else:
        user_vec = None

    # Compute cosine similarity between user input and course descriptions
    if user_vec is not None:
        # Assuming 'all_courses' is your DataFrame with course descriptions
        all_courses["Description"] = all_courses["Course Name"].apply(preprocess_text)
        all_courses["Description"] = all_courses["Description"].apply(
            lambda desc: np.mean([word_vectors[word] for word in desc.split() if word in word_vectors], axis=0)
        )

        all_courses["Similarity"] = all_courses["Description"].apply(
            lambda desc_vec: cosine_similarity([user_vec], [desc_vec])[0][0]
        )

        # Set a similarity threshold
        similarity_threshold = 0.6  # Adjust as needed

        # Filter courses based on similarity
        filtered_courses = all_courses[all_courses["Similarity"] > similarity_threshold]

        # Sort courses by similarity score
        recommended_courses = filtered_courses.sort_values(
            by="Similarity", ascending=False
        )

        # Display recommended courses to the user (customize columns as needed)
        recommended_courses = recommended_courses[[
            'University','Course Category', 'Course Name', 'Course Code', '2023 Planned Intake',
            'Range of Net 2023 JAE ELR2B2', 'Cutoff 2023', 'Cutoff 2022',
            'Cutoff 2021', 'Cutoff 2020', 'Cutoff 2019', 'Cutoff 2018',
            'Cutoff 2017', 'Cutoff 2016', 'Cutoff 2015', 'Cutoff 2014',
            'Mean Cutoff', 'Median Cutoff', 'Mode Cutoff', 'Similarity'
        ]]

        display(recommended_courses)
    else:
        print("No relevant words in user input.")


# This cell will run generate the output

In [19]:
import pandas as pd
import ipywidgets as widgets

# Define input widgets
elr2b2_input = widgets.IntSlider(
    value=15,
    min=5,
    max=35,
    step=-1,
    description='',
    layout=widgets.Layout(width='200px')
)

elr2b2_label = widgets.Label("ELR2B2 (O Levels Score):")
elr2b2_box = widgets.VBox([elr2b2_label, elr2b2_input])

cca_score_input = widgets.IntSlider(
    value=0,
    min=0,
    max=2,
    step=1,
    description='',
    layout=widgets.Layout(width='200px')
)

cca_score_label = widgets.Label("CCA Score:")
cca_score_box = widgets.VBox([cca_score_label, cca_score_input])

interest_of_studies_input = widgets.SelectMultiple(
    options=["Computing","Aviation", 'Computer & Information Technology', 'Business', "AI", "Arts", "Management", "Data Science", "Mechanical", "Game", "Design"],
    value=['Computer & Information Technology'],
    description='Interest of\nStudies:'
)

# Define a submit button
submit_button = widgets.Button(description='Submit')

# Define output widgets
output_table = widgets.Output()

# Function to handle the submit button click event
def on_submit_button_click(b, similarity=0.5):
    with output_table:
        # Clear previous output
        output_table.clear_output()
        
        # Validate ELR2B2 input
        elr2b2_value = elr2b2_input.value
        if not (5 <= elr2b2_value <= 35):
            Javascript("alert('Invalid ELR2B2 input. Please enter a value between 5 and 35.');")
            return
        
        # Validate CCA Score input
        cca_score_value = cca_score_input.value
        if not (0 <= cca_score_value <= 2):
            Javascript("alert('Invalid CCA Score input. Please enter a value between 0 and 2.');")
            return

        # Calculate user score
        user_score = elr2b2_value - cca_score_value
        
        all_filtered_courses = pd.DataFrame(columns=all_courses.columns)
        #print(all_courses.columns)

        for interest in list(interest_of_studies_input.value):
            # Filter the recommended courses based on user score
            filtered_courses = filter_courses(all_courses, interest, user_score, 0.3)
            all_filtered_courses = pd.concat([all_filtered_courses, filtered_courses])
        
        # Display the filtered courses as a DataFrame
        if not all_filtered_courses.empty:
            # Assuming 'all_filtered_courses' is your DataFrame
            all_filtered_courses.rename(columns={'University': 'Polytechnic'}, inplace=True)
            all_filtered_courses.rename(columns={'Course Category': 'School Name'}, inplace=True)
            

            # Columns to consider
            cutoff_columns = ['Cutoff 2014', 'Cutoff 2015', 'Cutoff 2016', 'Cutoff 2017', 'Cutoff 2018',
                              'Cutoff 2019', 'Cutoff 2020', 'Cutoff 2021', 'Cutoff 2022', 'Cutoff 2023']
            #print(cutoff_columns)
            # Extract the years (column names) and their corresponding cutoff values
            # Plot course cutoff scores over the years
            display(all_filtered_courses[
                ['Polytechnic','School Name', 'Course Name', 'Course Code', '2023 Planned Intake',
                'Range of Net 2023 JAE ELR2B2', 'Cutoff 2023','Mean Cutoff', 'Median Cutoff', 'Mode Cutoff', 
                'Course URL','Course Description', 'Career Prospect']])
            for index, row in all_filtered_courses.iterrows():

                years = [year.split()[-1] for year in cutoff_columns]
                cutoff_values = [row[year] for year in cutoff_columns]

                # Create a line graph for each course
                plt.figure(figsize=(12, 6))
                plt.plot(years, cutoff_values, marker='o', linestyle='-', color='b')
                plt.title(f"Cutoff Scores Over the Years for {row['Course Name']}")
                plt.xlabel("Year")
                plt.ylabel("Cutoff Score")
                plt.grid(True)
                plt.show()
        else:
            print("No courses match your criteria.")

# Set the function to be called when the submit button is clicked
submit_button.on_click(on_submit_button_click)

# Display the input widgets and submit button
input_widgets = widgets.HBox([elr2b2_box, cca_score_box, interest_of_studies_input, submit_button])
display(input_widgets)

# Display the output table
display(output_table)


Output()